# Preparação do ambiente

## Bibliotecas

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import unidecode
import warnings 

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.decomposition import TruncatedSVD
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from string import punctuation

In [2]:
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore')

## Constantes e funções auxiliares

In [3]:
def tokenizer(doc):
    doc = doc.lower()
    doc = unidecode.unidecode(doc)
    
    # uniformizar termos
    patterns = [('wi\s?.?fi', 'wifi'), 
                ('2,4', '2.4'), 
                ('5,8', '5.8')]
    for pattern, repl in patterns:
        doc = re.sub(pattern, repl, doc)
    
    stop_words = stopwords.words('portuguese')
    stop_words.extend(stopwords.words('english'))
    stop_words.extend(list(punctuation))
    # stopwords específicas do domínio
    stop_words.extend(['cm', 'feature', 'features', 'informações', 'itens', 'leve', 'list', 'nulo', 'package', 'pacote', 'pacotes', 'recurso', 'tamanho', 'ver', 'unidades', 'fio', 'universal'])
    # cores mais comumns
    stop_words.extend(['preto', 'cinza', 'branco', 'rosa', 'vermelho', 'laranja', 'amarelo', 'verde', 'azul', 'roxo', 'marrom'])
    # remover da lista de stopwords a palavra sem para formar o bigrama "sem fio", que pode ser relevante para o domínio
    # stop_words.remove('sem') 
    
    tokens = [token for token in word_tokenize(doc) if token not in stop_words]
    return tokens

def doc_cleaner(doc):
    tokens = tokenizer(doc)
    return ' '.join(tokens) 

# Carga e prepação dos dados

In [4]:
file_cat_sample = '../datasets/cat_to_label_sample.xlsx'
df_cat_sample = pd.read_excel(file_cat_sample)

df_cat_sample = df_cat_sample.iloc[:,-2:]
df_cat_sample = df_cat_sample.dropna()
df_cat_sample.columns = ['doc', 'labeled_category']

map_telecom_product = {'Antena': 1,  
                       'Baterias (eletrônicos em geral)': 0, 
                       'Cabo (em geral)': 0, 
                       'Cabo de rede': 1, 
                       'Câmera sem fio': 1, 
                       'Carregador de celular': 1,
                       'Carregador portátil (powerbank)': 1,
                       'Carregador sem fio': 1, 
                       'Carregadores e fontes (eletrônicos em geral)': 0, 
                       'Cartão de memória': 0,
                       'Desktop/Notebook': 0, 
                       'Drone': 1, 
                       'Extensor de vídeo VGA/HDMI': 0, 
                       'Fone de ouvido sem fio': 1, 
                       'Hub Switch': 1, 
                       'Impressora e cartuchos': 0,
                       'Microfone sem fio': 1, 
                       'Modem 3G/4G/WiFi': 1, 
                       'Outros': 0, 
                       'Reforçador de sinal de celular': 1, 
                       'Roteador/Reforçador WiFi': 1, 
                       'Smart TV': 1,
                       'Smartwatch': 1, 
                       'Tablets': 1, 
                       'Telefone celular': 1, 
                       'Transceptor de Radiação Restrita': 1, 
                       'Transceptor de RF (HF/VHF/UHF)': 1, 
                       'Transceptor/Conversor SFP': 1, 
                       'TV Box': 1}

map_rf_transmitter = {'Antena': 1,  
                      'Baterias (eletrônicos em geral)': 0, 
                      'Cabo (em geral)': 0, 
                      'Cabo de rede': 1, 
                      'Câmera sem fio': 2, 
                      'Carregador de celular': 1,
                      'Carregador portátil (powerbank)': 1,
                      'Carregador sem fio': 1, 
                      'Carregadores e fontes (eletrônicos em geral)': 0, 
                      'Cartão de memória': 0,
                      'Desktop/Notebook': 0, 
                      'Drone': 2, 
                      'Extensor de vídeo VGA/HDMI': 0, 
                      'Fone de ouvido sem fio': 2, 
                      'Hub Switch': 1, 
                      'Impressora e cartuchos': 0,
                      'Microfone sem fio': 2, 
                      'Modem 3G/4G/WiFi': 2, 
                      'Outros': 0,                        
                      'Reforçador de sinal de celular': 2, 
                      'Roteador/Reforçador WiFi': 2, 
                      'Smart TV': 2,
                      'Smartwatch': 2, 
                      'Tablets': 2, 
                      'Telefone celular': 2, 
                      'Transceptor de Radiação Restrita': 2, 
                      'Transceptor de RF (HF/VHF/UHF)': 2, 
                      'Transceptor/Conversor SFP': 1, 
                      'TV Box': 2}

df_cat_sample['telecom_product'] = df_cat_sample['labeled_category'].map(map_telecom_product)
df_cat_sample['rf_transmitter'] = df_cat_sample['labeled_category'].map(map_rf_transmitter)

map_category_to_int = {category:i for i,category in enumerate(df_cat_sample.labeled_category.unique())}
map_category_to_name = {i:category for category,i in map_category_to_int.items()}
df_cat_sample['category'] = df_cat_sample['labeled_category'].map(map_category_to_int)

df_cat_sample['clean_doc'] = df_cat_sample.doc.map(doc_cleaner)
columns_to_keep = ['doc', 'clean_doc', 'labeled_category', 'telecom_product', 'rf_transmitter', 'category']
df_cat_sample = df_cat_sample[columns_to_keep]

df_cat_sample

,doc,clean_doc,labeled_category,telecom_product,rf_transmitter,category
0,"Antena WiFi omnidirecional 2 peças, antena 4DB...",antena wifi omnidirecional 2 pecas antena 4dbi...,Antena,1,1,0
1,"Kesoto 2 peças macho 2,4 GHz 5 Ghz 5,8 Ghz ant...",kesoto 2 pecas macho 2.4 ghz 5 ghz 5.8 ghz ant...,Antena,1,1,0
2,Hemobllo Antenas 5 Unidades módulo transceptor...,hemobllo antenas 5 modulo transceptor antena b...,Antena,1,1,0
3,3 peças placa de rede sem fio WiFi antena rote...,3 pecas placa rede wifi antena roteador 2.4g/5...,Antena,1,1,0
4,Angoily 2 Unidades Substituição Da Antena Bast...,angoily 2 substituicao antena bastoes cola rec...,Antena,1,1,0
...,...,...,...,...,...,...
2086,Gps para Bike Xoss G Bluetooth Mtb Speed,gps bike xoss g bluetooth mtb speed,Transceptor de Radiação Restrita,1,2,25
2087,"Módulo AUX Bluetooth para carro, 11Pin Bluetoo...",modulo aux bluetooth carro 11pin bluetooth 5.0...,Transceptor de Radiação Restrita,1,2,25
2088,Operitacx Batente De Porta Protetor De Parada ...,operitacx batente porta protetor parada piso p...,Outros,0,0,18
2089,"Tecido de Faraday de 43 x 39 polegadas, tecido...",tecido faraday 43 x 39 polegadas tecido farada...,Outros,0,0,18


# Análise

## Classificação binária

### Produtos de telecomunicações x Outros

In [89]:
X = df_cat_sample['doc']
y = df_cat_sample['telecom_product']
target_names = ['Produtos comuns', 'Produtos de telecomunicações']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=724)

In [90]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenizer)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [91]:
clf_dummy = DummyClassifier()
clf_dummy.fit(X_train_tfidf,y_train)
predicted = clf_dummy.predict(X_test_tfidf)

print('Accuracy of Dummy classifier on training set: {:.2f}'
     .format(clf_dummy.score(X_train, y_train)))
print('Accuracy of Dummy classifier on test set: {:.2f}'
     .format(clf_dummy.score(X_test, y_test)))

print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of Dummy classifier on training set: 0.55
Accuracy of Dummy classifier on test set: 0.58
                              precision    recall  f1-score   support

             Produtos comuns       0.58      1.00      0.73       303
Produtos de telecomunicações       0.00      0.00      0.00       220

                    accuracy                           0.58       523
                   macro avg       0.29      0.50      0.37       523
                weighted avg       0.34      0.58      0.43       523



In [92]:
clf_sgd = SGDClassifier(loss='log_loss', penalty='l2',
                        alpha=1e-3, random_state=42,
                        max_iter=5, tol=None)

clf_sgd.fit(X_train_tfidf, y_train)
predicted = clf_sgd.predict(X_test_tfidf)

print('Accuracy of SGD classifier on training set: {:.3f}'
     .format(clf_sgd.score(X_train_tfidf, y_train)))
print('Accuracy of SGD classifier on test set: {:.3f}'
     .format(clf_sgd.score(X_test_tfidf, y_test)))
print('AUC of SGD classifier on test set: {:.3f}'
     .format(roc_auc_score(y_test,clf_sgd.predict_proba(X_test_tfidf)[:,1])))
print()
print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of SGD classifier on training set: 0.936
Accuracy of SGD classifier on test set: 0.860
AUC of SGD classifier on test set: 0.931

                              precision    recall  f1-score   support

             Produtos comuns       0.85      0.91      0.88       303
Produtos de telecomunicações       0.87      0.79      0.83       220

                    accuracy                           0.86       523
                   macro avg       0.86      0.85      0.85       523
                weighted avg       0.86      0.86      0.86       523



$Precision = \frac{TP}{TP+FP}$ quanto maior, menor a quantidade de falsos positivos

$Recall = \frac{TP}{TP+FN}$ quanto maior, menor a quantidade de falsos negativos

In [93]:
from sklearn.linear_model import LogisticRegression

clf_lgr = LogisticRegression()

clf_lgr.fit(X_train_tfidf, y_train)
predicted = clf_lgr.predict(X_test_tfidf)

print('Accuracy of SGD classifier on training set: {:.3f}'
     .format(clf_lgr.score(X_train_tfidf, y_train)))
print('Accuracy of SGD classifier on test set: {:.3f}'
     .format(clf_lgr.score(X_test_tfidf, y_test)))
print('AUC of SGD classifier on test set: {:.3f}'
     .format(roc_auc_score(y_test,clf_lgr.predict_proba(X_test_tfidf)[:,1])))
print()
print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of SGD classifier on training set: 0.948
Accuracy of SGD classifier on test set: 0.866
AUC of SGD classifier on test set: 0.936

                              precision    recall  f1-score   support

             Produtos comuns       0.87      0.91      0.89       303
Produtos de telecomunicações       0.87      0.80      0.83       220

                    accuracy                           0.87       523
                   macro avg       0.87      0.86      0.86       523
                weighted avg       0.87      0.87      0.87       523



In [53]:
# from sklearn.metrics import accuracy_score, make_scorer

# # The scorers can be either one of the predefined metric strings or a scorer
# # callable, like the one returned by make_scorer
# scoring = {"AUC": "roc_auc"}

# pipe = Pipeline([('tfidf',TfidfVectorizer(tokenizer=tokenizer)),('clf',SGDClassifier())])

# params = {'tfidf__ngram_range': [(1,1),(1,2),(1,3)], 
#           'clf__loss': ['hinge', 'log_loss', 'modified_huber'], 
#           'clf__penalty': ['l1', 'l2'],
#           'clf__alpha': [1e-1,1e-2,1e-3,1e-4,1e-5], 'clf__random_state': [42],
#           'clf__max_iter': [5,20,50,100], 'clf__tol': [None]}

# # initialize
# clf = SGDClassifier()
# gd = GridSearchCV(pipe,params,scoring=scoring,refit='AUC')
# # fit
# gd.fit(X_train,y_train)
# gd.best_params_

{'clf__alpha': 1e-05,
 'clf__loss': 'hinge',
 'clf__max_iter': 100,
 'clf__penalty': 'l2',
 'clf__random_state': 42,
 'clf__tol': None,
 'tfidf__ngram_range': (1, 2)}

In [54]:
gd.best_score_

0.9521058748268423

In [97]:
pipe_sgd = Pipeline([('tfidf',TfidfVectorizer(tokenizer=tokenizer,ngram_range=(1,2))),
                     ('clf', SGDClassifier(alpha=1e-5, 
                                           loss='hinge', 
                                           max_iter=100, 
                                           penalty='l2', 
                                           random_state=42, 
                                           tol=None))])
pipe_sgd.fit(X_train,y_train)
predicted = pipe_sgd.predict(X_test)

print('Accuracy of SGD classifier on training set: {:.3f}'
     .format(pipe_sgd.score(X_train, y_train)))
print('Accuracy of SGD classifier on test set: {:.3f}'
     .format(pipe_sgd.score(X_test, y_test)))
print('AUC of SGD classifier on test set: {:.3f}'
     .format(roc_auc_score(y_test,pipe_sgd.decision_function(X_test))))
print()
print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of SGD classifier on training set: 1.000
Accuracy of SGD classifier on test set: 0.889
AUC of SGD classifier on test set: 0.954

                              precision    recall  f1-score   support

             Produtos comuns       0.92      0.89      0.90       303
Produtos de telecomunicações       0.85      0.89      0.87       220

                    accuracy                           0.89       523
                   macro avg       0.89      0.89      0.89       523
                weighted avg       0.89      0.89      0.89       523



In [98]:
# modelo anterior aparentemente overfitting, reduzir alpha
pipe_sgd = Pipeline([('tfidf',TfidfVectorizer(tokenizer=tokenizer,ngram_range=(1,2))),
                     ('clf', SGDClassifier(alpha=1e-3, 
                                           loss='hinge', 
                                           max_iter=100, 
                                           penalty='l2', 
                                           random_state=42, 
                                           tol=None))])

pipe_sgd.fit(X_train,y_train)
predicted = pipe_sgd.predict(X_test)

print('Accuracy of SGD classifier on training set: {:.3f}'
     .format(pipe_sgd.score(X_train, y_train)))
print('Accuracy of SGD classifier on test set: {:.3f}'
     .format(pipe_sgd.score(X_test, y_test)))
print('AUC of SGD classifier on test set: {:.3f}'
     .format(roc_auc_score(y_test,pipe_sgd.decision_function(X_test))))
print()
print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of SGD classifier on training set: 0.980
Accuracy of SGD classifier on test set: 0.885
AUC of SGD classifier on test set: 0.948

                              precision    recall  f1-score   support

             Produtos comuns       0.90      0.90      0.90       303
Produtos de telecomunicações       0.87      0.86      0.86       220

                    accuracy                           0.89       523
                   macro avg       0.88      0.88      0.88       523
                weighted avg       0.89      0.89      0.89       523



### Produtos transmissores de RF x Outros

In [100]:
map_rf_transmitter = {0: 0, 1: 0, 2: 1}
X = df_cat_sample['doc']
y = df_cat_sample['rf_transmitter'].map(map_rf_transmitter)

target_names = ['Produtos comuns', 'Produtos transmissores de RF']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=724)

pipe_sgd_rf = Pipeline([('tfidf',TfidfVectorizer(tokenizer=tokenizer,ngram_range=(1,2))),
                        ('clf', SGDClassifier(alpha=1e-3, 
                                              loss='hinge', 
                                              max_iter=100, 
                                              penalty='l2', 
                                              random_state=42, 
                                              tol=None))])

pipe_sgd_rf.fit(X_train,y_train)
predicted = pipe_sgd_rf.predict(X_test)

print('Accuracy of SGD classifier on training set: {:.3f}'
     .format(pipe_sgd_rf.score(X_train, y_train)))
print('Accuracy of SGD classifier on test set: {:.3f}'
     .format(pipe_sgd_rf.score(X_test, y_test)))
print('AUC of SGD classifier on test set: {:.3f}'
     .format(roc_auc_score(y_test,pipe_sgd_rf.decision_function(X_test))))
print()
print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of SGD classifier on training set: 0.984
Accuracy of SGD classifier on test set: 0.902
AUC of SGD classifier on test set: 0.962

                              precision    recall  f1-score   support

             Produtos comuns       0.92      0.92      0.92       331
Produtos transmissores de RF       0.87      0.87      0.87       192

                    accuracy                           0.90       523
                   macro avg       0.89      0.90      0.90       523
                weighted avg       0.90      0.90      0.90       523



## Classificação em 3 classes

In [101]:
X = df_cat_sample['doc']
y = df_cat_sample['rf_transmitter']
target_names = ['Produtos comuns', 'Produtos de telecomunicações', 'Transmissores de RF']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=724)

In [102]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenizer)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [103]:
clf_dummy = DummyClassifier()
clf_dummy.fit(X_train_tfidf,y_train)
predicted = clf_dummy.predict(X_test_tfidf)

print('Accuracy of Dummy classifier on training set: {:.2f}'
     .format(clf_dummy.score(X_train_tfidf, y_train)))
print('Accuracy of Dummy classifier on test set: {:.2f}'
     .format(clf_dummy.score(X_test_tfidf, y_test)))

print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of Dummy classifier on training set: 0.55
Accuracy of Dummy classifier on test set: 0.58
                              precision    recall  f1-score   support

             Produtos comuns       0.58      1.00      0.73       303
Produtos de telecomunicações       0.00      0.00      0.00        28
         Transmissores de RF       0.00      0.00      0.00       192

                    accuracy                           0.58       523
                   macro avg       0.19      0.33      0.24       523
                weighted avg       0.34      0.58      0.43       523



In [104]:
clf_sgd = SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-3, random_state=42,
                        max_iter=5, tol=None)

clf_sgd.fit(X_train_tfidf, y_train)
predicted = clf_sgd.predict(X_test_tfidf)

print('Accuracy of SGD classifier on training set: {:.2f}'
     .format(clf_sgd.score(X_train_tfidf, y_train)))
print('Accuracy of SGD classifier on test set: {:.2f}'
     .format(clf_sgd.score(X_test_tfidf, y_test)))

print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of SGD classifier on training set: 0.96
Accuracy of SGD classifier on test set: 0.87
                              precision    recall  f1-score   support

             Produtos comuns       0.87      0.92      0.89       303
Produtos de telecomunicações       1.00      0.21      0.35        28
         Transmissores de RF       0.86      0.89      0.88       192

                    accuracy                           0.87       523
                   macro avg       0.91      0.67      0.71       523
                weighted avg       0.88      0.87      0.86       523



In [105]:
clf_lgr = LogisticRegression()

clf_lgr.fit(X_train_tfidf, y_train)
predicted = clf_lgr.predict(X_test_tfidf)

print('Accuracy of SGD classifier on training set: {:.2f}'
     .format(clf_lgr.score(X_train_tfidf, y_train)))
print('Accuracy of SGD classifier on test set: {:.2f}'
     .format(clf_lgr.score(X_test_tfidf, y_test)))

print(classification_report(y_test, predicted, target_names=target_names))

Accuracy of SGD classifier on training set: 0.94
Accuracy of SGD classifier on test set: 0.85
                              precision    recall  f1-score   support

             Produtos comuns       0.84      0.93      0.88       303
Produtos de telecomunicações       1.00      0.07      0.13        28
         Transmissores de RF       0.87      0.84      0.85       192

                    accuracy                           0.85       523
                   macro avg       0.90      0.61      0.62       523
                weighted avg       0.86      0.85      0.83       523

